In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [3]:
sqladb=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',names=['Age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','>50K, <=50K'])
# 1)  Select 10 records from the adult sqladb
print(pysqldf("SELECT *   from sqladb LIMIT 10 ;"))

   Age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   
5   37            Private  284582     Masters             14   
6   49            Private  160187         9th              5   
7   52   Self-emp-not-inc  209642     HS-grad              9   
8   31            Private   45781     Masters             14   
9   42            Private  159449   Bachelors             13   

           marital-status          occupation    relationship    race  \
0           Never-married        Adm-clerical   Not-in-family   White   
1      Married-civ-spouse     Exec-managerial         Husband   White   
2                Divorced   Handlers-cleaners   Not-in-family   White   
3  

In [4]:
# 2)  show me the average hours per week for all the mens #working in private sector
def trimAllColumns(df):
  trimStrings = lambda x:x.strip() if type(x) is str else x
  return df.applymap(trimStrings)

sqladb = trimAllColumns(sqladb)

sqladb.columns = sqladb.columns.str.replace('-','_')

print(pysqldf("SELECT sex, workclass,avg(hours_per_week)  from sqladb where sex='Male' and workclass='Private' ;"))


    sex workclass  avg(hours_per_week)
0  Male   Private            42.221226


In [5]:
# 3) show me the frequency table of education, occupation and relationship seperately

In [6]:
print(pysqldf("SELECT education, count(education) FROM sqladb group by education;"))

       education  count(education)
0           10th               933
1           11th              1175
2           12th               433
3        1st-4th               168
4        5th-6th               333
5        7th-8th               646
6            9th               514
7     Assoc-acdm              1067
8      Assoc-voc              1382
9      Bachelors              5355
10     Doctorate               413
11       HS-grad             10501
12       Masters              1723
13     Preschool                51
14   Prof-school               576
15  Some-college              7291


In [7]:
print(pysqldf("SELECT occupation, count(occupation) FROM sqladb group by occupation;"))

           occupation  count(occupation)
0                   ?               1843
1        Adm-clerical               3770
2        Armed-Forces                  9
3        Craft-repair               4099
4     Exec-managerial               4066
5     Farming-fishing                994
6   Handlers-cleaners               1370
7   Machine-op-inspct               2002
8       Other-service               3295
9     Priv-house-serv                149
10     Prof-specialty               4140
11    Protective-serv                649
12              Sales               3650
13       Tech-support                928
14   Transport-moving               1597


In [8]:
print(pysqldf("SELECT relationship, count(relationship) FROM sqladb group by relationship;"))

     relationship  count(relationship)
0         Husband                13193
1   Not-in-family                 8305
2  Other-relative                  981
3       Own-child                 5068
4       Unmarried                 3446
5            Wife                 1568


In [9]:
# 4) Are their any people who are married , working in private sector and have master's degree
print(print(pysqldf("SELECT *   from sqladb WHERE marital_status in ('Married-civ-spouse','Married-spouse-absent','Married-AF-spouse') and workclass='Private' and education ='Masters' ;")))

     Age workclass  fnlwgt education  education_num         marital_status  \
0     37   Private  284582   Masters             14     Married-civ-spouse   
1     33   Private  202051   Masters             14     Married-civ-spouse   
2     76   Private  124191   Masters             14     Married-civ-spouse   
3     31   Private   99928   Masters             14     Married-civ-spouse   
4     35   Private  138992   Masters             14     Married-civ-spouse   
5     34   Private  142897   Masters             14     Married-civ-spouse   
6     62   Private  270092   Masters             14     Married-civ-spouse   
7     41   Private  445382   Masters             14     Married-civ-spouse   
8     33   Private  208405   Masters             14     Married-civ-spouse   
9     49   Private  192776   Masters             14     Married-civ-spouse   
10    51   Private  410114   Masters             14     Married-civ-spouse   
11    31   Private  118710   Masters             14     Married-

In [10]:
# 5) What is the average , maximum, minimum age of people working in different sector
print(pysqldf("SELECT avg(age),max(age),min(age),workclass from sqladb group by workclass ;"))

    avg(age)  max(age)  min(age)         workclass
0  40.960240        90        17                 ?
1  42.590625        90        17       Federal-gov
2  41.751075        90        17         Local-gov
3  20.571429        30        17      Never-worked
4  36.797585        90        17           Private
5  46.017025        84        17      Self-emp-inc
6  44.969697        90        17  Self-emp-not-inc
7  39.436055        81        17         State-gov
8  47.785714        72        19       Without-pay


In [11]:
# 6) Calculate age distribution by country
print(pysqldf("SELECT count(age) as age , native_country from sqladb group by native_country ;"))

      age              native_country
0     583                           ?
1      19                    Cambodia
2     121                      Canada
3      75                       China
4      59                    Columbia
5      95                        Cuba
6      70          Dominican-Republic
7      28                     Ecuador
8     106                 El-Salvador
9      90                     England
10     29                      France
11    137                     Germany
12     29                      Greece
13     64                   Guatemala
14     44                       Haiti
15      1          Holand-Netherlands
16     13                    Honduras
17     20                        Hong
18     13                     Hungary
19    100                       India
20     43                        Iran
21     24                     Ireland
22     73                       Italy
23     81                     Jamaica
24     62                       Japan
25     18   

In [12]:
# 7) Compute a new Column as Net-Capital Gain from the two columns 'capital-gain' and 'capital loss'
sqladb["Net_Captial_Loss"] =  sqladb["capital_gain"].astype(int) - sqladb["capital_loss"].astype(int) 
print(sqladb["Net_Captial_Loss"])

0         2174
1            0
2            0
3            0
4            0
5            0
6            0
7            0
8        14084
9         5178
10           0
11           0
12           0
13           0
14           0
15           0
16           0
17           0
18           0
19           0
20           0
21           0
22           0
23       -2042
24           0
25           0
26           0
27           0
28           0
29           0
         ...  
32531        0
32532        0
32533        0
32534        0
32535        0
32536        0
32537        0
32538    15020
32539        0
32540        0
32541        0
32542        0
32543        0
32544        0
32545        0
32546        0
32547        0
32548     1086
32549        0
32550        0
32551        0
32552        0
32553        0
32554        0
32555        0
32556        0
32557        0
32558        0
32559        0
32560    15024
Name: Net_Captial_Loss, Length: 32561, dtype: int32
